# Introduction
Building on the model visualization of Mol et al., the acetone production pathway is added 

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite
from os.path import dirname, join, pardir

In [2]:
data_dir = "/Users/phivri/Documents/GitHub/MoGeoModel"
model = cobra.io.read_sbml_model(join(data_dir,'Models/p-thermo_acetone_anaerobic.xml'))

Using license file /Users/phivri/gurobi.lic
Academic license - for non-commercial use only - expires 2022-07-23


In [3]:
#Compare to E.coli core
#model_e_coli = cameo.load_model('iML1515')
#builder = Builder(model=model_e_coli, map_name = 'e_coli_core.Core metabolism')
#builder

In [4]:
#here I can build the map on the e. coli core backbone, taking the initial changes
Builder(model=model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'))

Builder()

__Additional changes__ Modifying the map also lets us see some changes that need to be made. 

For example, this strain has been shown to produce formate upon fermentation, and so we need a formate transport reaction that is currently lacking. (data from Tang et al. 2009, doi 10.1002/bit.22181) Bioinformatically, a formate-proton symporter has been identified and so this will be added. 

In [5]:
model.reactions.PTAr.lower_bound = -1000
model.reactions.ACt.lower_bound = -1000
model.reactions.EX_ac_e.lower_bound = -1000

In [6]:
medium = model.medium
medium["EX_o2_e"] = 1000.0
medium["EX_ac_e"] = 10.0
medium["EX_sucr_e"] = 0.0
medium["EX_glc__D_e"] = 0.0
model.medium = medium

In [7]:
solution = model.optimize()
Builder(model = model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

Builder(reaction_data={'IDPh': 10.436720513765293, 'CAT': -1.0282152926375218e-08, 'PDHam1hi': 0.0, 'HYDA': 0.…

In [9]:
Y_XS_max = solution.fluxes.EX_Biomass_e/solution.fluxes.EX_ac_e
ratio = 0.8
Y_XS_max

-0.013191307305315297

In [12]:
MaxGrowth = model.problem.Constraint(
    model.reactions.EX_Biomass_e.flux_expression - Y_XS_max * ratio * model.reactions.EX_ac_e.flux_expression,
    lb=0,
    ub=1000)
model.add_cons_vars(MaxGrowth)

model.objective = 'EX_act_e'

solution = model.optimize()
#Builder(model = model, map_json = join(data_dir,'Models/p-thermo_acetone_map.json'), reaction_data = solution.fluxes.to_dict())

ContainerAlreadyContains: Container '<optlang.container.Container object at 0x7f9f7e066e80>' already contains an object with name 'add884e2-31bb-11ec-84e4-a683e7763947'.